<a href="https://colab.research.google.com/github/Abhijith-Nagarajan/CS_546_Project/blob/Abhijith_NER_TM/notebooks/RE_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
package_directory = r'/content/gdrive/MyDrive/CS_546_Project_Dependencies/lib/python3.10/site-packages'

In [ ]:
import sys

In [ ]:
sys.path.append(package_directory)

In [ ]:
import os

In [ ]:
import copy

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
from spacy import displacy
import en_core_sci_md
import re
import torch

In [ ]:
from datasets import load_dataset, Dataset

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoModelForSeq2SeqLM, AutoModelForCausalLM, AutoModel, TrainingArguments, Trainer, IntervalStrategy, EarlyStoppingCallback

In [ ]:
model_path = "/content/gdrive/MyDrive/CS_546_Project_Dependencies/BioRED_Results/checkpoint-118"

In [ ]:
rebel_model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
rebel_model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50272, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50272, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    

In [ ]:
rebel_tokenizer = AutoTokenizer.from_pretrained(model_path)

In [ ]:
pubmed_inference_data = pd.read_json("/content/research_text_pubmed_new.json")

In [ ]:
pubmed_inference_data.head()

In [ ]:
pubmed_inference_data.shape

(103, 473)

In [ ]:
pubmed_inference_filtered = pubmed_inference_data[['title','abstract']]

In [ ]:
pubmed_inference_filtered

,title,abstract
0,Genetic and phenotypic diversity of NHE6 mutat...,abstract
1,LSD1-mediated epigenetic modification is requi...,abstract TAL1 is a critical transcription fa...
2,TFBSshape: a motif database for DNA shape feat...,abstract Transcription factor binding sites ...
3,Histone H3K27ac separates active from poised e...,abstract Developmental programs are controll...
4,Novel C12orf65mutations in patients with axona...,abstract
...,...,...
98,MARS: improving multiple circular sequence ali...,abstract
99,JACUSA: site-specific identification of RNA ed...,abstract
100,RIblast: an ultrafast RNA–RNA interaction pred...,abstract
101,TIRR regulates 53BP1 by masking its histone me...,NaN


In [ ]:
def preprocess_text(text: str):
  '''
  The aim of this function is to preprocess text by converting it to lowercase, and punctuations.

  This is not required.
  '''
  text = text.lower()
  text = re.sub(r",\s?",' ', text)
  text = re.sub(r'\.\s*', ' ', text)
  return text

In [ ]:
def filter_text(text:str):
    try:
        if type(text) != str:
            return ''
        else:
            if 'abstract' not in text:
                return text
            else:
                text = text.replace('abstract ', '')
                return text
    except:
        print(f'Error text: {text}...{type(text)}')

In [ ]:
pubmed_inference_filtered['abstract_updated'] = pubmed_inference_filtered['abstract'].apply(lambda row: filter_text(row))

<ipython-input-31-3f59f4ba2418>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pubmed_inference_filtered['abstract_updated'] = pubmed_inference_filtered['abstract'].apply(lambda row: filter_text(row))


In [ ]:
#pubmed_inference_filtered['abstract_updated'] = pubmed_inference_filtered['abstract_updated'].apply(lambda row: preprocess_text(row))

In [ ]:
pubmed_inference_filtered['abstract_updated'] = pubmed_inference_filtered['abstract_updated'].str.strip()

In [ ]:
pubmed_inference_filtered['input'] = pubmed_inference_filtered['title']+" "+pubmed_inference_filtered['abstract_updated']

In [ ]:
from spacy.tokens import Doc, Span
from fastcoref import FCoref

In [ ]:
#nlp = spacy.load('en_core_web_sm')
nlp_scispacy = spacy.load('en_core_sci_md')

In [ ]:
fcoref_model = FCoref(device=device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/819 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/393 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


pytorch_model.bin:   0%|          | 0.00/362M [00:00<?, ?B/s]

In [ ]:
def core_logic_part(document: Doc, coref: list[int], resolved: list[str], mention_span: Span):
    final_token = document[coref[1]]
    if final_token.tag_ in ["PRP$", "POS"]:
        resolved[coref[0]] = mention_span.text + "'s" + final_token.whitespace_
    else:
        resolved[coref[0]] = mention_span.text + final_token.whitespace_
    for i in range(coref[0] + 1, coref[1] + 1):
        resolved[i] = ""
    return resolved

In [ ]:
def get_span_noun_indices(doc: Doc, cluster: list[list[int]]) -> list[int]:
    try:
      spans = [doc[span[0]:span[1]+1] for span in cluster]
      spans_pos = [[token.pos_ for token in span] for span in spans]
      span_noun_indices = [i for i, span_pos in enumerate(spans_pos)
          if any(pos in span_pos for pos in ['NOUN', 'PROPN'])]
      return span_noun_indices
    except Exception as e:
      print(f'Error received while getting span noun indices. {e}. Returning empty list')
      return []

In [ ]:
def get_cluster_head(doc: Doc, cluster: list[list[int]], noun_indices: list[int]):
    head_idx = noun_indices[0]
    head_start, head_end = cluster[head_idx]
    head_span = doc[head_start:head_end+1]
    return head_span, [head_start, head_end]

In [ ]:
def is_containing_other_spans(span: list[int], all_spans: list[list[int]]):
    return any([s[0] >= span[0] and s[1] <= span[1] and s != span for s in all_spans])

In [ ]:
def improved_replace_corefs(document, clusters):
    resolved = list(tok.text_with_ws for tok in document)
    all_spans = [span for cluster in clusters for span in cluster]  # flattened list of all spans

    for cluster in clusters:
        noun_indices = get_span_noun_indices(document, cluster)

        if noun_indices:
            mention_span, mention = get_cluster_head(document, cluster, noun_indices)

            for coref in cluster:
                if coref != mention and not is_containing_other_spans(coref, all_spans):
                    core_logic_part(document, coref, resolved, mention_span)

    return "".join(resolved)

In [ ]:
def get_fast_cluster_spans(doc, clusters):
    fast_clusters = []
    for cluster in clusters:
        new_group = []
        for tuple in cluster:
            #print(type(tuple), tuple)
            (start, end) = tuple
            #print("start, end", start, end)
            span = doc.char_span(start, end)
            #print('span', span.start, span.end)
            new_group.append([span.start, span.end-1])
        fast_clusters.append(new_group)
    return fast_clusters

In [ ]:
def get_fastcoref_clusters(doc, text, fcoref_model):
  try:
    preds = fcoref_model.predict(texts=[text])
    fast_clusters = preds[0].get_clusters(as_strings=False)
    fast_cluster_spans = get_fast_cluster_spans(doc, fast_clusters)
  except Exception as e:
    print(f'Error received while getting coreference clusters. {e}. Returning original text')
    return text
  return fast_cluster_spans

In [ ]:
def perform_coreference_resolution(text):
    doc = nlp_scispacy(text)
    clusters = get_fastcoref_clusters(doc, text, fcoref_model)
    coref_text = improved_replace_corefs(doc, clusters)
    return coref_text

In [ ]:
pubmed_inference_filtered['input_coref'] = pubmed_inference_filtered['input'].apply(perform_coreference_resolution)

def preprocess_test_data(examples):
    model_inputs = rebel_tokenizer(
        examples["input_coref"],  # Source text
        max_length=512,
        truncation=True,
        padding="max_length"
    )
    return model_inputs

In [ ]:
pubmed_inference_filtered['input_coref']

,input_coref
0,Genetic and phenotypic diversity of NHE6 mutat...
1,LSD1-mediated epigenetic modification is requi...
2,TFBSshape: a motif database for DNA shape feat...
3,Histone H3K27ac separates active from poised e...
4,Novel C12orf65mutations in patients with axona...
...,...
98,MARS: improving multiple circular sequence ali...
99,JACUSA: site-specific identification of RNA ed...
100,RIblast: an ultrafast RNA–RNA interaction pred...
101,TIRR regulates 53BP1 by masking 53BP1's histon...


In [ ]:
pubmed_inference_filtered.to_csv('pubmed_inderence_filtered.csv')

In [ ]:
def extract_triplets(decoded_string):
    decoded_str_copy = decoded_string
    if '<s>' in decoded_string:
      decoded_str_copy = ''.join(decoded_string.split('<s>')[1:])
      decoded_str_copy = decoded_str_copy.split('</s>')[0]

    triplets = []
    parts = decoded_str_copy.split("<triplet>")

    for part in parts[1:]:  # Skip the first empty split
        # Extract the head entity (the first part before <subj>)
        head, *relations = part.split("<subj>")
        head = head.strip()

        # Parse each <subj> and <obj> pair
        for relation in relations:
            if "<obj>" in relation:
                subj, obj = relation.split("<obj>", 1)
                triplets.append((head, subj.strip(), obj.strip()))

    return triplets

In [ ]:
def perform_inference(text):
  extracted_triplets = []
  for sentence in text.split('. '):
    inputs = rebel_tokenizer(sentence, return_tensors="pt", truncation=True, max_length=512).to(device)
    outputs = rebel_model.generate(**inputs)
    decoded_output = rebel_tokenizer.decode(outputs[0], skip_special_tokens=False)
    triplets = list(set(extract_triplets(decoded_output)))
    if len(triplets)==0:
      continue
    else:
      extracted_triplets.extend(triplets)

  return extracted_triplets

In [ ]:
pubmed_inference_filtered['triplets'] = pubmed_inference_filtered['input_coref'].apply(perform_inference)

<ipython-input-68-620b74cd144d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pubmed_inference_filtered['triplets'] = pubmed_inference_filtered['input_coref'].apply(perform_inference)


In [ ]:
pubmed_inference_filtered.to_csv('pubmed_inference_filtered_with_triplets.csv')

In [ ]:
aggregated_triplets = list(set().union(*pubmed_inference_filtered['triplets']))

In [ ]:
aggregated_triplets

[('p53', 'MDM2', 'Association'),
 ('RNA polymerase II', '5S ribosomal RNA', 'Association'),
 ('LSD1', 'p4.2 promoter', 'Positive_Correlation'),
 ('Preeclampsia', 'hyperinflammatory', 'Association'),
 ('DNA damage', 'H2A', 'Positive_Correlation'),
 ('Ezh2', 'H3K27 methyltransferase', 'Association'),
 ('p53', 'cancer', 'Positive_Correlation'),
 ('TMPDB', 'web site', 'instance of'),
 ('FRDA', 'Friedreich’s PsyNetataxia', 'Association'),
 ('erythropoiesis', 'GATA-1', 'Association'),
 ('Gfi-1b', 'hematopoietic factor', 'subclass of'),
 ('Ebola Virus Infection', 'ZMapp', 'Positive_Correlation'),
 ('GATA-1', 'hematopoietic factor', 'subclass of'),
 ('Pta1 Scaffold Protein', 'Pta1 Scaffold Protein', 'Association'),
 ('pineal region', 'spinal cord', 'connects with'),
 ('5S ribosomal RNA', 'RNA polymerase II', 'Association'),
 ('cell cycle progression', 'DNA replication', 'Positive_Correlation'),
 ('TAL1', 'LSD1', 'Positive_Correlation'),
 ('Eco1', 'sister chromatid cohesion', 'Association'),
 (

In [ ]:
with open("aggregated_triplets.txt", "w") as f:
    for triplet in aggregated_triplets:
        f.write(f"{triplet}\n")

In [ ]:
import pandas as pd
df = pd.read_csv('clustered_data.csv')

In [ ]:
df.head()

,abstract,introduction,results,discussion,methods,cluster_id,title
0,Germline pathogenic variants associated with ...,"With the rise of the genomic era, a growing n...",pCPS genes with monoallelic risk phenotypes s...,Here we have explored the theory that AD(LoF)...,"In order to ensure reproducibility the full, ...",0,NaN
1,NaN,"Globally, colorectal cancer (CRC) is the thir...",Establishment of CRC organoids from primary t...,Current strategies for precision medicine are...,Establishment of patient-derived organoid cul...,4,A functional personalised oncology approach ag...
2,NaN,The advances in sequencing technology have im...,Selection of 33 patients with suspected hered...,"In this study, we demonstrated that TAS-LRS c...",Ethics approval and consent to participateThe...,3,Assessing the efficacy of target adaptive samp...
3,NaN,Breast carcinoma is the leading cause of canc...,3.1. Patient demographics and clinicopatholog...,"We addressed two goals in this study. First, ...",NaN,2,Genomic disparity impacts variant classificati...
4,NaN,Characterising the genomic landscape of somat...,NaN,NaN,NaN,0,COSMIC: a curated database of somatic variants...


In [ ]:
df_cluster = df.loc[df['cluster_id'] == 0]
df_cluster_1 = df.loc[df['cluster_id'] == 1]
df_cluster_2 = df.loc[df['cluster_id'] == 2]
df_cluster_3 = df.loc[df['cluster_id'] == 3]
df_cluster_4 = df.loc[df['cluster_id'] == 4]

In [ ]:
df['cluster_id'].unique()

array([0, 4, 3, 2, 1])

In [ ]:
df_cluster.rename(columns={'abstract':'some', 'introduction': 'abstract'}, inplace=True)

<ipython-input-30-6ba4ab6b41a8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cluster.rename(columns={'abstract':'some', 'introduction': 'abstract'}, inplace=True)


In [ ]:
df_cluster_1.rename(columns={'abstract':'some', 'introduction': 'abstract'}, inplace=True)
df_cluster_2.rename(columns={'abstract':'some', 'introduction': 'abstract'}, inplace=True)
df_cluster_3.rename(columns={'abstract':'some', 'introduction': 'abstract'}, inplace=True)
df_cluster_4.rename(columns={'abstract':'some', 'introduction': 'abstract'}, inplace=True)

<ipython-input-39-c03283b3da61>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cluster_2.rename(columns={'abstract':'some', 'introduction': 'abstract'}, inplace=True)
<ipython-input-39-c03283b3da61>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cluster_3.rename(columns={'abstract':'some', 'introduction': 'abstract'}, inplace=True)
<ipython-input-39-c03283b3da61>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cluster_4.r

In [ ]:
df_cluster.head()

,some,abstract,results,discussion,methods,cluster_id,title
0,Germline pathogenic variants associated with ...,"With the rise of the genomic era, a growing n...",pCPS genes with monoallelic risk phenotypes s...,Here we have explored the theory that AD(LoF)...,"In order to ensure reproducibility the full, ...",0,NaN
4,NaN,Characterising the genomic landscape of somat...,NaN,NaN,NaN,0,COSMIC: a curated database of somatic variants...
5,NaN,NaN,A framework for measuring enrichment of germl...,We noted an enrichment of rare germline varia...,Tumor sequencing data from PCAWGThe final con...,0,Systematic analysis of Mendelian disease-assoc...
11,NaN,Prostate cancer (PCa) is characterised by sub...,Known risk alleles in Black South Africans wi...,"In this study, motivated by limited studies h...",Ethics approvals and recruitmentWritten infor...,0,Prostate cancer genetic risk and associated ag...
13,Head and neck squamous cell carcinomas (HNSCC...,Head and neck squamous cell carcinoma (HNSCC)...,NaN,NaN,NaN,0,Saliva as a potential non-invasive liquid biop...


In [ ]:
df_cluster.shape

(7, 7)

#### Wrapping inference as a class

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
package_directory = r'/content/gdrive/MyDrive/CS_546_Project_Dependencies/lib/python3.10/site-packages'

In [ ]:
import sys
sys.path.append(package_directory)

In [ ]:
import os
import torch
import pandas as pd
import spacy
import torch
from fastcoref import FCoref
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [ ]:
from spacy.tokens import Doc, Span

In [ ]:
class RelationExtractor:
    def __init__(self, nlp_scispacy, fcoref_model, rebel_model, rebel_tokenizer, device, data=None):
      """
      Initialize the RelationExtractor class with models for coreference, triplet extraction, and data handling.
      """
      self.nlp_scispacy = nlp_scispacy
      self.fcoref_model = fcoref_model
      self.rebel_model = rebel_model
      self.rebel_tokenizer = rebel_tokenizer
      self.device = device
      self.data = data

    def filter_data(self):
      """
      Filter and clean the abstract text.
      """
      if self.data is None:
          raise ValueError("No data loaded. Please load data using first.")

      self.data['title'] = self.data['title'].fillna('')
      df_filtered = self.data[['title', 'abstract']]
      df_filtered['abstract_updated'] = df_filtered['abstract'].apply(self.filter_text)
      df_filtered['input'] = ( df_filtered['title'] + " " + df_filtered['abstract_updated'])
      self.data = df_filtered

    ### Utility Methods ###
    def get_fast_cluster_spans(self, doc, clusters):
      """
      Convert fast coreference model output clusters into spans.
      """
      fast_clusters = []
      for cluster in clusters:
          new_group = []
          for span_tuple in cluster:
              (start, end) = span_tuple
              span = doc.char_span(start, end)
              if span:
                  new_group.append([span.start, span.end - 1])
          fast_clusters.append(new_group)
      return fast_clusters

    def get_fastcoref_clusters(self, doc, text):
      """
      Extract coreference clusters using the FastCoref model.
      """
      try:
          preds = self.fcoref_model.predict(texts=[text])
          fast_clusters = preds[0].get_clusters(as_strings=False)
          fast_cluster_spans = self.get_fast_cluster_spans(doc, fast_clusters)
      except Exception as e:
          print(f'Error while getting coreference clusters: {e}')
          return []
      return fast_cluster_spans

    def core_logic_part(self, document: Doc, coref: list[int], resolved: list[str], mention_span: Span):
      """
      Replace a coreference mention with the cluster's head mention.
      """
      final_token = document[coref[1]]
      if final_token.tag_ in ["PRP$", "POS"]:
          resolved[coref[0]] = mention_span.text + "'s" + final_token.whitespace_
      else:
          resolved[coref[0]] = mention_span.text + final_token.whitespace_
      for i in range(coref[0] + 1, coref[1] + 1):
          resolved[i] = ""
      return resolved

    def get_span_noun_indices(self, doc: Doc, cluster: list[list[int]]) -> list[int]:
      """
      Get indices of noun spans within a coreference cluster.
      """
      try:
          spans = [doc[span[0]:span[1] + 1] for span in cluster]
          spans_pos = [[token.pos_ for token in span] for span in spans]
          span_noun_indices = [
              i for i, span_pos in enumerate(spans_pos)
              if any(pos in span_pos for pos in ['NOUN', 'PROPN'])
          ]
          return span_noun_indices
      except Exception as e:
          print(f'Error while getting span noun indices: {e}')
          return []

    def get_cluster_head(self, doc: Doc, cluster: list[list[int]], noun_indices: list[int]):
      """
      Determine the head mention for a coreference cluster.
      """
      head_idx = noun_indices[0]
      head_start, head_end = cluster[head_idx]
      head_span = doc[head_start:head_end + 1]
      return head_span, [head_start, head_end]

    def is_containing_other_spans(self, span: list[int], all_spans: list[list[int]]):
      """
      Check if a span contains other spans within a cluster.
      """
      return any([s[0] >= span[0] and s[1] <= span[1] and s != span for s in all_spans])

    def filter_text(self, text):
      """
      Filter and clean the abstract text.
      """
      try:
          if type(text) != str:
              return ''
          if 'abstract' in text:
              return text.replace('abstract ', '').strip()
          return text.strip()
      except Exception as e:
          print(f'Error filtering text: {e}')
          return ''

    ### Coreference Resolution Methods ###
    def improved_replace_corefs(self, document, clusters):
      """
      Replace coreferences with their head mentions in the document.
      """
      resolved = list(tok.text_with_ws for tok in document)
      all_spans = [span for cluster in clusters for span in cluster]

      for cluster in clusters:
          noun_indices = self.get_span_noun_indices(document, cluster)

          if noun_indices:
              mention_span, mention = self.get_cluster_head(document, cluster, noun_indices)

              for coref in cluster:
                  if coref != mention and not self.is_containing_other_spans(coref, all_spans):
                      self.core_logic_part(document, coref, resolved, mention_span)

      return "".join(resolved)

    def perform_coreference_resolution(self, text):
      """
      Perform coreference resolution on the input text.
      """
      try:
          doc = self.nlp_scispacy(text)
          clusters = self.get_fastcoref_clusters(doc, text)
          coref_text = self.improved_replace_corefs(doc, clusters)
          return coref_text
      except Exception as e:
          print(f"Error in coreference resolution: {e}")
          return text

    ### Triplet Extraction Methods ###
    def extract_triplets(self, decoded_string):
      """
      Extract triplets from the decoded string.
      """
      decoded_str_copy = decoded_string
      if '<s>' in decoded_string:
          decoded_str_copy = ''.join(decoded_string.split('<s>')[1:])
          decoded_str_copy = decoded_str_copy.split('</s>')[0]

      triplets = []
      parts = decoded_str_copy.split("<triplet>")

      for part in parts[1:]:
          head, *relations = part.split("<subj>")
          head = head.strip()

          for relation in relations:
              if "<obj>" in relation:
                  subj, obj = relation.split("<obj>", 1)
                  triplets.append((head, subj.strip(), obj.strip()))

      return triplets

    def perform_inference(self, text):
      """
      Perform inference to extract triplets from the input text.
      """
      try:
          extracted_triplets = []
          for sentence in text.split('. '):
              inputs = self.rebel_tokenizer(
                  sentence, return_tensors="pt", truncation=True, max_length=512
              ).to(self.device)
              outputs = self.rebel_model.generate(**inputs)
              decoded_output = self.rebel_tokenizer.decode(outputs[0], skip_special_tokens=False)
              triplets = list(set(self.extract_triplets(decoded_output)))
              if triplets:
                  extracted_triplets.extend(triplets)
          return extracted_triplets
      except Exception as e:
          print(f"Error during inference: {e}")
          return []

    ### Data Handling and Aggregation Methods ###
    def load_data(self, file_path):
      """
      Load and preprocess data from a JSON file.
      """

      if self.data is None:
        df = pd.read_json(file_path)
        df['title'] = df['title'].fillna('')
        df_filtered = df[['title', 'abstract']]
        df_filtered['abstract_updated'] = df_filtered['abstract'].apply(self.filter_text)
        df_filtered['input'] = ( df_filtered['title'] + " " + df_filtered['abstract_updated'])

        self.data = df_filtered

        '''
        else:
          print('Data already loaded')
          self.data['title'] = self.data['title'].fillna('')
          df_filtered = self.data[['title', 'abstract']]
          df_filtered['abstract_updated'] = df_filtered['abstract'].apply(self.filter_text)
          df_filtered['input'] = ( df_filtered['title'] + " " + df_filtered['abstract_updated'])
          self.data = df_filtered

        return self.data
        '''
      #return pubmed_inference_filtered

    def process_coreference_resolution(self):
      """
      Process coreference resolution for the loaded data.
      """
      if self.data is None:
          raise ValueError("No data loaded. Please load data using `load_data` method.")
      self.data['input_coref'] = self.data['input'].apply(self.perform_coreference_resolution)

    def process_triplet_extraction(self):
      """
      Process triplet extraction for the loaded data.
      """
      if 'input_coref' not in self.data.columns:
          raise ValueError("Coreference resolution not performed. Please run `process_coreference_resolution` first.")
      self.data['triplets'] = self.data['input_coref'].apply(self.perform_inference)

    def aggregate_triplets(self):
      """
      Aggregate all extracted triplets into a single set.
      """
      if 'triplets' not in self.data.columns:
          raise ValueError("Triplet extraction not performed. Please run `process_triplet_extraction` first.")
      aggregated_triplets = list(set().union(*self.data['triplets']))
      return aggregated_triplets

    def write_triplets_to_file(self, aggregated_triplets, file_path):
      """
      Write aggregated triplets to a text file.
      """
      with open(file_path, "w") as f:
          for triplet in aggregated_triplets:
              f.write(f"{triplet}\n")


In [ ]:
model_path = "/content/gdrive/MyDrive/CS_546_Project_Dependencies/BioRED_Results/checkpoint-118"

In [ ]:
file_path = '/content/research_text_pubmed_new.json'

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
rebel_model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

In [ ]:
rebel_model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50272, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50272, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    

In [ ]:
rebel_tokenizer = AutoTokenizer.from_pretrained(model_path)

In [ ]:
nlp_scispacy = spacy.load('en_core_sci_md')

/usr/local/lib/python3.10/dist-packages/spacy/language.py:2195: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]


In [ ]:
fcoref_model = FCoref(device=device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/819 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/393 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


pytorch_model.bin:   0%|          | 0.00/362M [00:00<?, ?B/s]

In [ ]:
relation_extractor = RelationExtractor(nlp_scispacy, fcoref_model, rebel_model, rebel_tokenizer, device, df_cluster)

In [ ]:
relation_extractor_1 = RelationExtractor(nlp_scispacy, fcoref_model, rebel_model, rebel_tokenizer, device, df_cluster_1)

In [ ]:
relation_extractor_2 = RelationExtractor(nlp_scispacy, fcoref_model, rebel_model, rebel_tokenizer, device, df_cluster_2)
relation_extractor_3 = RelationExtractor(nlp_scispacy, fcoref_model, rebel_model, rebel_tokenizer, device, df_cluster_3)
relation_extractor_4 = RelationExtractor(nlp_scispacy, fcoref_model, rebel_model, rebel_tokenizer, device, df_cluster_4)

##### Extractor - Cluster 0

In [ ]:
relation_extractor.filter_data()

In [ ]:
relation_extractor.process_coreference_resolution()

In [ ]:
relation_extractor.process_triplet_extraction()

In [ ]:
aggregated_triplets = relation_extractor.aggregate_triplets()

In [ ]:
aggregated_triplets

In [ ]:
relation_extractor.write_triplets_to_file(aggregated_triplets, "aggregated_triplets_cluster0.txt")

##### Extractor - Cluster 1

In [ ]:
relation_extractor_1.filter_data()

In [ ]:
relation_extractor_1.process_coreference_resolution()

In [ ]:
relation_extractor_1.process_triplet_extraction()

In [ ]:
aggregated_triplets_1 = relation_extractor_1.aggregate_triplets()

In [ ]:
relation_extractor_1.write_triplets_to_file(aggregated_triplets_1, "aggregated_triplets_cluster1.txt")

##### Extractor - Cluster 2

In [ ]:
relation_extractor_2.filter_data()

In [ ]:
relation_extractor_2.process_coreference_resolution()

In [ ]:
relation_extractor_2.process_triplet_extraction()

In [ ]:
aggregated_triplets_2 = relation_extractor_2.aggregate_triplets()

In [ ]:
relation_extractor_2.write_triplets_to_file(aggregated_triplets_2, "aggregated_triplets_cluster2.txt")

##### Extractor - Cluster 3

In [ ]:
relation_extractor_3.filter_data()

In [ ]:
relation_extractor_3.process_coreference_resolution()

In [ ]:
relation_extractor_3.process_triplet_extraction()

In [ ]:
aggregated_triplets_3 = relation_extractor_3.aggregate_triplets()

In [ ]:
relation_extractor_3.write_triplets_to_file(aggregated_triplets_3, "aggregated_triplets_cluster3.txt")

##### Extractor - Cluster 4

In [ ]:
relation_extractor_4.filter_data()

In [ ]:
relation_extractor_4.process_coreference_resolution()

In [ ]:
relation_extractor_4.process_triplet_extraction()

In [ ]:
aggregated_triplets_4 = relation_extractor_4.aggregate_triplets()

In [ ]:
relation_extractor_4.write_triplets_to_file(aggregated_triplets_4, "aggregated_triplets_cluster4.txt")

#### Experimental

In [ ]:
text5 = "Jennifer Doudna discovered the CRISPR-Cas9 system. She later won a Nobel Prize."
text5_a = "Jennifer Doudna and Emmanuelle Charpentier discovered the CRISPR-Cas9 system. She later won a Nobel Prize."

In [ ]:
text6 = "Chandragupta Maurya was the founder of the Maurya Empire and dynasty, based in Magadha. His rise to power began after the death of Alaxander in 323 BCE."
text6_a = "Chandragupta Maurya was the founder of the Maurya Empire and dynasty. His rise to power began after the death of Alaxander in 323 BCE."
text6_b = "Chandragupta Maurya's rise to power began after the death of Alaxander in 323 BCE."
text6_c = "Chandragupta Maurya's rise to power began in 323 BCE."
text6_d = "Chandragupta Maurya's rise to power began after he won the war."

In [ ]:
inputs = rebel_tokenizer(text5_a, return_tensors="pt", max_length=1024, truncation=False).to(device)

In [ ]:
outputs = rebel_model.generate(**inputs)

In [ ]:
extracted_text = rebel_tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
extracted_text

' CRISPR-Cas9  Jennifer Doudna  discoverer or inventor  Emmanuelle Charpentier  discoverer or'

In [ ]:
test_str = pubmed_inference_filtered['input_coref'][1]

In [ ]:
test_str1 = 'LSD1-mediated epigenetic modification is required for TAL1 function and hematopoiesis TAL1 is a critical transcription factor required for hematopoiesis'

In [ ]:
extracted_triplets = []
for sentence in test_str1.split('. '):
  inputs = rebel_tokenizer(sentence, return_tensors="pt", truncation=True, max_length=512).to(device)
  outputs = rebel_model.generate(**inputs)
  decoded_output = rebel_tokenizer.decode(outputs[0], skip_special_tokens=False)
  triplets = list(set(extract_triplets(decoded_output)))
  if len(triplets)==0:
    continue
  else:
    extracted_triplets.extend(triplets)

In [ ]:
extracted_triplets

[('TAL1', 'epigenetic modification', 'Association'),
 ('hematopoiesis', 'TAL1', 'Association'),
 ('TAL1', 'hematopoiesis', 'Association')]

In [ ]:
inputs

{'input_ids': tensor([[    0, 10463,   495,   134,    12, 43728, 44323, 15557, 29685,    16,
          1552,    13,   255,  2118,   134,  5043,     8,    37,  9244,  1517,
           139,   918,   354,  1437,  1437,   255,  2118,   134,    16,    10,
          2008, 37118,  3724,  1552,    13,    37,  9244,  1517,   139,   918,
           354,     4,   635,     6, 32819, 13157,  1258,     9,    63,  1940,
           747,  3315,     7,   255,  3551, 28837,     4,  5994,     8,   141,
            63, 37118,   337,  1713,    32, 13588,   148,    37,  9244,  1517,
           139,   918,   354,  1189,     7,    28,  4873,     4,  1398,     6,
            52,   311,    14,   255,  2118,   134,    16,  3059,    19, 33945,
          1264,  4410, 30298,  3175, 34003,  8200,   784,  2459,   833,    12,
         14175,  4410, 30298,  3175,   112,    36, 10463,   495,   134,   238,
          4979,   134, 11052, 17725, 37118,  3724,  2731, 13447,   368,    36,
          8739,   500,  4923,   238, 3

In [ ]:
outputs = rebel_model.generate(**inputs)

In [ ]:
decoded_output = rebel_tokenizer.decode(outputs[0], skip_special_tokens=False)

In [ ]:
decoded_output

'<s><triplet> LSD1 <subj> TAL1 <obj> Association <triplet> TAL1 <subj> LSD1 <obj> Association</s>'

In [ ]:
triplets = set(extract_triplets(decoded_output))

In [ ]:
triplets

set()

In [ ]:
perform_inference(test_str)

KeyboardInterrupt: 

pubmed_dataset = Dataset.from_pandas(pubmed_inference_filtered[['input_coref']])

pubmed_dataset

tokenized_pubmed_dataset = pubmed_dataset.map(preprocess_test_data, batched=True)

class RelationExtractor:
    def __init__(self, nlp_scispacy, fcoref_model, rebel_model, rebel_tokenizer, device):
        """
        Initialize the RelationExtractor class with models for coreference, triplet extraction, and data handling.
        """
        self.nlp_scispacy = nlp_scispacy
        self.fcoref_model = fcoref_model
        self.rebel_model = rebel_model
        self.rebel_tokenizer = rebel_tokenizer
        self.device = device
        self.data = None

    def filter_text(self, text):
        """
        Filter and clean the abstract text.
        """
        try:
            if type(text) != str:
                return ''
            if 'abstract' in text:
                return text.replace('abstract ', '').strip()
            return text.strip()
        except Exception as e:
            print(f'Error filtering text: {e}')
            return ''

    ### Data Reading Methods ###
    def load_data(self, file_path):
        """
        Load and preprocess data from a JSON file.
        """
        pubmed_inference_data = pd.read_json(file_path)
        pubmed_inference_filtered = pubmed_inference_data[['title', 'abstract']]
        pubmed_inference_filtered['abstract_updated'] = pubmed_inference_filtered['abstract'].apply(self.filter_text)
        pubmed_inference_filtered['input'] = (
            pubmed_inference_filtered['title'] + " " + pubmed_inference_filtered['abstract_updated']
        )
        self.data = pubmed_inference_filtered
        return pubmed_inference_filtered

    ### Coreference Resolution Methods ###
    def core_logic_part(self, document: Doc, coref: list[int], resolved: list[str], mention_span: Span):
        final_token = document[coref[1]]
        if final_token.tag_ in ["PRP$", "POS"]:
            resolved[coref[0]] = mention_span.text + "'s" + final_token.whitespace_
        else:
            resolved[coref[0]] = mention_span.text + final_token.whitespace_
        for i in range(coref[0] + 1, coref[1] + 1):
            resolved[i] = ""
        return resolved

    def get_span_noun_indices(self, doc: Doc, cluster: list[list[int]]) -> list[int]:
        try:
            spans = [doc[span[0]:span[1] + 1] for span in cluster]
            spans_pos = [[token.pos_ for token in span] for span in spans]
            span_noun_indices = [
                i for i, span_pos in enumerate(spans_pos)
                if any(pos in span_pos for pos in ['NOUN', 'PROPN'])
            ]
            return span_noun_indices
        except Exception as e:
            print(f'Error while getting span noun indices: {e}')
            return []

    def get_cluster_head(self, doc: Doc, cluster: list[list[int]], noun_indices: list[int]):
        head_idx = noun_indices[0]
        head_start, head_end = cluster[head_idx]
        head_span = doc[head_start:head_end + 1]
        return head_span, [head_start, head_end]

    def is_containing_other_spans(self, span: list[int], all_spans: list[list[int]]):
        return any([s[0] >= span[0] and s[1] <= span[1] and s != span for s in all_spans])

    def improved_replace_corefs(self, document, clusters):
        resolved = list(tok.text_with_ws for tok in document)
        all_spans = [span for cluster in clusters for span in cluster]

        for cluster in clusters:
            noun_indices = self.get_span_noun_indices(document, cluster)

            if noun_indices:
                mention_span, mention = self.get_cluster_head(document, cluster, noun_indices)

                for coref in cluster:
                    if coref != mention and not self.is_containing_other_spans(coref, all_spans):
                        self.core_logic_part(document, coref, resolved, mention_span)

        return "".join(resolved)

    def get_fast_cluster_spans(self, doc, clusters):
        fast_clusters = []
        for cluster in clusters:
            new_group = []
            for span_tuple in cluster:
                (start, end) = span_tuple
                span = doc.char_span(start, end)
                if span:
                    new_group.append([span.start, span.end - 1])
            fast_clusters.append(new_group)
        return fast_clusters

    def get_fastcoref_clusters(self, doc, text):
        try:
            preds = self.fcoref_model.predict(texts=[text])
            fast_clusters = preds[0].get_clusters(as_strings=False)
            fast_cluster_spans = self.get_fast_cluster_spans(doc, fast_clusters)
        except Exception as e:
            print(f'Error while getting coreference clusters: {e}')
            return text
        return fast_cluster_spans

    def perform_coreference_resolution(self, text):
        """
        Perform coreference resolution on the input text.
        """
        doc = self.nlp_scispacy(text)
        clusters = self.get_fastcoref_clusters(doc, text)
        coref_text = self.improved_replace_corefs(doc, clusters)
        return coref_text

    ### Triplet Extraction Methods ###
    def extract_triplets(self, decoded_string):
        """
        Extract triplets from the decoded string.
        """
        decoded_str_copy = decoded_string
        if '<s>' in decoded_string:
            decoded_str_copy = ''.join(decoded_string.split('<s>')[1:])
            decoded_str_copy = decoded_str_copy.split('</s>')[0]

        triplets = []
        parts = decoded_str_copy.split("<triplet>")

        for part in parts[1:]:
            head, *relations = part.split("<subj>")
            head = head.strip()

            for relation in relations:
                if "<obj>" in relation:
                    subj, obj = relation.split("<obj>", 1)
                    triplets.append((head, subj.strip(), obj.strip()))

        return triplets

    def perform_inference(self, text):
        """
        Perform inference to extract triplets from the input text.
        """
        extracted_triplets = []
        for sentence in text.split('. '):
            inputs = self.rebel_tokenizer(
                sentence, return_tensors="pt", truncation=True, max_length=512
            ).to(self.device)
            outputs = self.rebel_model.generate(**inputs)
            decoded_output = self.rebel_tokenizer.decode(outputs[0], skip_special_tokens=False)
            triplets = list(set(self.extract_triplets(decoded_output)))
            if triplets:
                extracted_triplets.extend(triplets)

        return extracted_triplets

    def aggregate_triplets(self, input_texts):
        """
        Aggregate triplets for a collection of texts.
        """
        aggregated_triplets = []
        
        for text in input_texts:
            coref_text = self.perform_coreference_resolution(text)
            triplets = self.perform_inference(coref_text)
            aggregated_triplets.extend(triplets)
        return list(set(aggregated_triplets))

    def write_triplets_to_file(self, aggregated_triplets, file_path):
        """
        Write aggregated triplets to a text file.
        """
        with open(file_path, "w") as f:
            for triplet in aggregated_triplets:
                f.write(f"{triplet}\n")
